Alright, I lied. One more exploratory plot - Hovmoller plot of different rainfall types, and then a summary plot showing the area average - essentially, showing seasonality.

In [1]:
import itertools
import matplotlib.pyplot as plt
import netCDF4 as nc
import numpy as np
import time
import os
import seaborn as sn
from shutil import copy

%matplotlib inline

from jd_matplotlib_tools import add_double_calendar_axis, mark_time_periods

In [2]:
ferretpath = "/Users/Siwen/Desktop/ferret/bin"
rdapath = "APHRO_ZH_front_025deg_V1101."

In [3]:
## start by loading all precipitation data for each year.
P = np.zeros((365,80,72,57))
P_band = np.zeros((365,80,72,57))
P_local = np.zeros((365,80,72,57))

for yr in range(1951,2008):
    
    #Build filename
    filename = ferretpath + '/' + rdapath + str(yr) + '.nc'
    print(filename),
    myf = nc.Dataset(filename, 'r')
    
    #Obtain fraction of different types of rainfall
    P[:,:,:,yr-1951] = myf['P'][:365,:,:]
    P_band[:,:,:,yr-1951] = myf['P_band1'][:365,:,:] + myf['P_band2'][:365,:,:]
    P_local[:,:,:,yr-1951] = myf['P_leftover'][:365,:,:]
    
    myf.close()
    
P_tuple = (P,P_band,P_local)
P_mean = [ np.mean(P_type, axis=3) for P_type in P_tuple ]

/Users/Siwen/Desktop/ferret/bin/APHRO_ZH_front_025deg_V1101.1951.nc
/Users/Siwen/Desktop/ferret/bin/APHRO_ZH_front_025deg_V1101.1952.nc
/Users/Siwen/Desktop/ferret/bin/APHRO_ZH_front_025deg_V1101.1953.nc
/Users/Siwen/Desktop/ferret/bin/APHRO_ZH_front_025deg_V1101.1954.nc
/Users/Siwen/Desktop/ferret/bin/APHRO_ZH_front_025deg_V1101.1955.nc
/Users/Siwen/Desktop/ferret/bin/APHRO_ZH_front_025deg_V1101.1956.nc
/Users/Siwen/Desktop/ferret/bin/APHRO_ZH_front_025deg_V1101.1957.nc
/Users/Siwen/Desktop/ferret/bin/APHRO_ZH_front_025deg_V1101.1958.nc
/Users/Siwen/Desktop/ferret/bin/APHRO_ZH_front_025deg_V1101.1959.nc
/Users/Siwen/Desktop/ferret/bin/APHRO_ZH_front_025deg_V1101.1960.nc
/Users/Siwen/Desktop/ferret/bin/APHRO_ZH_front_025deg_V1101.1961.nc
/Users/Siwen/Desktop/ferret/bin/APHRO_ZH_front_025deg_V1101.1962.nc
/Users/Siwen/Desktop/ferret/bin/APHRO_ZH_front_025deg_V1101.1963.nc
/Users/Siwen/Desktop/ferret/bin/APHRO_ZH_front_025deg_V1101.1964.nc
/Users/Siwen/Desktop/ferret/bin/APHRO_ZH_front_0

In [9]:
def smooth(P,dayrange,latrange):
    Psmooth=np.zeros(P.shape)
    
    ll=int((dayrange-1)/2)
    yy=int((latrange-1)/2)
    
    days = P.shape[0]
    lats = P.shape[1]
    
    for d in range(days):
        for j in range(lats):
            
            Psample = P.take(range(d-ll,d+ll+1), mode='wrap', axis=0)
            Ps = Psample[:,max(0,j-yy):min(days,j+yy+1)]
            Psmooth[d,j] = np.mean(np.mean(Ps))
    
    return Psmooth

In [12]:
## SMOOTHING ##
#smooth out the decadal precipitation means
#but, no latitudinal smoothing since rainfall already effectively smoothed in latitude
daysmth = 15
latsmth = 0

## for apparent memory reasons, have to actually write out each line.
P_smth = []
P_smth.append([ smooth(x,daysmth,latsmth) for x in P_mean[0] ])
P_smth.append([ smooth(x,daysmth,latsmth) for x in P_mean[1] ])
P_smth.append([ smooth(x,daysmth,latsmth) for x in P_mean[2] ])

KeyboardInterrupt: 

In [ ]:
# CREATE CLIMATOLOGY FIGURE

axlim = 2 #sets the maximum and minimum for colorbar.
mypalette = 'RdYlBu_r'

## Set up axis and title for the entire figure
fig, ax = plt.subplots(figsize=(11,8))
ax.set_title("Climatology by Rainfall Type", fontsize=18, y = 1.07, x=.43)
ax.set_axis_off()

## Subplot a) - Total rainfall
ax1 = fig.add_subplot(3,1,1)
plot1 = ax1.pcolormesh(P_smth[0].T, cmap = mypalette); 
#plt.contour(P_smth[0],1,colors = 'gray')
ax1.set_title("a) Mean Total Rainfall (mm day$^{-1}$)", fontsize=14, y=1.12, loc = 'left')
plt.ylabel('Latitude ($^\circ$)', fontsize=12, labelpad=0)
plt.yticks(np.linspace(0,80,6),np.linspace(20,40,6).astype(int))

#cbar1 = plt.colorbar(plot1, pad = 0.02)
#cbar1.set_ticks(np.linspace(-axlim,axlim,9))
#cbar1.set_ticklabels(['<-2',-1.5,-1,-.5,0,.5,1,1.5,'>2'])

add_double_calendar_axis()
mark_time_periods(time_periods, numbers = 'yes', box = 'no')


## Subplot b) - Banded rainfall
ax2 = fig.add_subplot(3,1,2)
plot2 = ax2.pcolormesh(P_smth[1].T, cmap = mypalette); 
#plt.contour(mk_band[0][1].T,1,colors = 'gray')
ax2.set_title("b) Mean Banded Rainfall (mm day$^{-1}$)", fontsize=14, y=1.12, loc = 'left')
plt.ylabel('Latitude ($^\circ$)', fontsize=12, labelpad=0)
plt.yticks(np.linspace(0,80,6),np.linspace(20,40,6).astype(int))

add_double_calendar_axis()
mark_time_periods(time_periods, numbers = 'yes', box = 'no')


## Subplot c) Local rainfall
ax3 = fig.add_subplot(3,1,3)
plot3 = ax3.pcolormesh(P_smth[2], cmap = mypalette); 
#plt.contour(freq_diff_mask_99[0][:,1:21].mask.T,1,colors = 'gray')
plt.ylabel('Latitude ($\!^\circ\!$)', fontsize=12, labelpad=0)
plt.yticks(np.linspace(0,20,6),np.linspace(20,40,6).astype(int))


ax3.set_title("c) Mean Local Rainfall (mm day$^{-1}$)", fontsize=14, y=1.12, loc = 'left')
add_double_calendar_axis()
mark_time_periods(time_periods, numbers = 'yes', box = 'no')